# Prepare seeds

In [ ]:
import os
import json
import gzip
import shutil
import requests
import zipfile
from pathlib import Path
from fhirpy import SyncFHIRClient

In [ ]:
SOURCE = "http://localhost:8080"
client = SyncFHIRClient(SOURCE)

In [ ]:
def download_and_unzip(url, headers, extract_to):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open("temp.zip", "wb") as f:
            f.write(response.content)

        with zipfile.ZipFile("temp.zip", 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        os.remove("temp.zip")
    else:
        print("Failed to download the file")
        return False
    return True

def filter_files(directory, exclude_strings):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(s in file for s in exclude_strings):
                os.remove(os.path.join(root, file))

def create_ndjson(input_dir, output_file):
    data = []
    for file_name in os.listdir(input_dir):
        with open(os.path.join(input_dir, file_name), 'r') as file:
            data.append(json.load(file))

    with open(output_file, 'w') as f:
        for entry in data:
            json.dump(entry, f)
            f.write('\n')

def gzip_file(input_file, output_file):
    with open(input_file, 'rb') as f_in:
        with gzip.open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

def move_file(src, dest):
    shutil.move(src, dest)

def clean_up(directory):
    shutil.rmtree(directory)

def to_aidbox_format(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
    
            with open(file_path, 'r') as file:
                data = json.load(file)
                result = client.execute("/$to-format/aidbox", method='post', data=data, params=None) 
                with open(file_path, 'w') as file:
                    json.dump(result["resource"], file, indent=4)

In [ ]:
url = "https://hl7.org.au/fhir/core/0.2.2-preview/examples.json.zip"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
include_strings = ["StructureDefinition", "ValueSet", "CapabilityStatement"]
extract_to = "extracted"
ndjson_file = "seeds.ndjson"
gzipped_file = "seeds.ndjson.gz"
destination_folder = "../zenproject/"

In [ ]:
if download_and_unzip(url, headers, extract_to):
    filter_files(extract_to, include_strings)
    to_aidbox_format(extract_to)
    create_ndjson(extract_to, ndjson_file)
    gzip_file(ndjson_file, gzipped_file)
    move_file(gzipped_file, Path(destination_folder) / gzipped_file)
    clean_up(extract_to)
    os.remove(ndjson_file)